In [1]:
import neutromeratio
from openmmtools.constants import kB
from simtk import unit
import numpy as np
import pickle
import mdtraj as md
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import torch
from neutromeratio.parameter_gradients import FreeEnergyCalculator
from neutromeratio.constants import kT, device, exclude_set
from glob import glob

# job idx
idx = int(1)
# where to write the results
base_path = str('/home/mwieder/')
env = str('droplet')
diameter_in_angstrom = int(18)
#######################

mode = 'forward'

# read in exp results, smiles and names
exp_results = pickle.load(open('../data/exp_results.pickle', 'rb'))

# name of the system
protocoll = []
for name in sorted(exp_results):
    if name in exclude_set:
        continue
    protocoll.append(name)

name = protocoll[idx-1]
print(name)

# don't change - direction is fixed for all runs
#################
t1_smiles = exp_results[name]['t1-smiles']
t2_smiles = exp_results[name]['t2-smiles']

exp_results[name]['energy']

_ColormakerRegistry()

[utils.py:141 - _init_num_threads()] NumExpr defaulting to 8 threads.


SAMPLmol2


-6.1

In [3]:
(-12.497892722203273 * kT).in_units_of(unit.kilocalorie_per_mole)

Quantity(value=-7.450768522014049, unit=kilocalorie/mole)

In [ ]:
import neutromeratio
from openmmtools.constants import kB
from simtk import unit
import numpy as np
import pickle
import mdtraj as md
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import torch
from neutromeratio.parameter_gradients import FreeEnergyCalculator
from neutromeratio.constants import kT, device, exclude_set
from glob import glob

def parse_lambda_from_dcd_filename(dcd_filename, env):
    return float(dcd_filename[:dcd_filename.find(f"_in_{env}")].split('_')[-1])


# job idx
idx = int(1)
# where to write the results
base_path = str('/home/mwieder/')
env = str('droplet')
diameter_in_angstrom = int(18)
#######################

mode = 'forward'

# read in exp results, smiles and names
exp_results = pickle.load(open('../data/exp_results.pickle', 'rb'))

# name of the system
protocoll = []
for name in sorted(exp_results):
    if name in exclude_set:
        continue
    protocoll.append(name)

name = protocoll[idx-1]
print(name)

# don't change - direction is fixed for all runs
#################
t1_smiles = exp_results[name]['t1-smiles']
t2_smiles = exp_results[name]['t2-smiles']


# generate both rdkit mol
tautomer = neutromeratio.Tautomer(name=name, initial_state_mol=neutromeratio.generate_rdkit_mol(t1_smiles), final_state_mol=neutromeratio.generate_rdkit_mol(t2_smiles), nr_of_conformations=20)
if mode == 'forward':
    tautomer.perform_tautomer_transformation_forward()
elif mode == 'reverse':
    tautomer.perform_tautomer_transformation_reverse()
else:
    raise RuntimeError('No tautomer reaction direction was specified.')

if env == 'droplet':
    tautomer.add_droplet(tautomer.hybrid_topology, 
                                tautomer.hybrid_coords, 
                                diameter=diameter_in_angstrom * unit.angstrom,
                                restrain_hydrogens=True,
                                file=f"{base_path}/{name}/{name}_in_droplet_{mode}.pdb")
    print('Nr of atoms: {}'.format(len(tautomer.ligand_in_water_atoms)))
    atoms = tautomer.ligand_in_water_atoms
    top = tautomer.ligand_in_water_topology
else:
    atoms = tautomer.hybrid_atoms
    top = tautomer.hybrid_topology

# define the alchemical atoms
alchemical_atoms=[tautomer.hybrid_hydrogen_idx_at_lambda_1, tautomer.hybrid_hydrogen_idx_at_lambda_0]


# extract hydrogen donor idx and hydrogen idx for from_mol
model = neutromeratio.ani.LinearAlchemicalDualTopologyANI(alchemical_atoms=alchemical_atoms, adventure_mode=True)
model = model.to(device)
torch.set_num_threads(2)

# perform initial sampling
energy_function = neutromeratio.ANI1_force_and_energy(
                                        model = model,
                                        atoms = atoms,
                                        mol = None,
                                        )


for r in tautomer.ligand_restraints:
    energy_function.add_restraint(r)

for r in tautomer.hybrid_ligand_restraints:
    energy_function.add_restraint(r)

if env == 'droplet':

    tautomer.add_COM_for_hybrid_ligand(np.array([diameter_in_angstrom/2, diameter_in_angstrom/2, diameter_in_angstrom/2]) * unit.angstrom)

    for r in tautomer.solvent_restraints:
        energy_function.add_restraint(r)

    for r in tautomer.com_restraints:
        energy_function.add_restraint(r)


# get steps inclusive endpoints
# and lambda values in list
dcds = glob(f"{base_path}/{name}/*.dcd")

lambdas = []
ani_trajs = []
energies = []

for dcd_filename in dcds:
    lam = parse_lambda_from_dcd_filename(dcd_filename, env)
    lambdas.append(lam)
    traj = md.load_dcd(dcd_filename, top=top)
    ani_trajs.append(traj)  
    f = open(f"{base_path}/{name}/{name}_lambda_{lam:0.4f}_energy_in_{env}_{mode}.csv", 'r')  
    tmp_e = []
    for e in f:
        tmp_e.append(float(e))
    f.close()
    energies.append(np.array(tmp_e))

In [2]:
snapshots = {}
K = len(ani_trajs)
for lam, traj in zip(lambdas, ani_trajs):

    #equil, g = detectEquilibration(self.potential_energy_trajs[i])[:2]
    #thinning = int(g)
    #if len(traj[equil::thinning]) > max_snapshots_per_window:
        # what thinning will give me len(traj[equil::thinning]) == max_snapshots_per_window?
    #    thinning = int((len(traj) - equil) / max_snapshots_per_window)

    new_snapshots = list(traj[0:200:10].xyz * unit.nanometer)
    snapshots[lam] = new_snapshots

In [3]:
def calculate_stddev(snapshots):
    lambda0_e_b_stddev = [energy_function.calculate_energy(x, lambda_value=0.0) for x in tqdm(snapshots)]
    lambda1_e_b_stddev = [energy_function.calculate_energy(x, lambda_value=1.0) for x in tqdm(snapshots)]

    # extract endpoint stddev
    lambda0_stddev = [stddev/kT for stddev in [e_b_stddev[2] for e_b_stddev in lambda0_e_b_stddev]]
    lambda1_stddev = [stddev/kT for stddev in [e_b_stddev[2] for e_b_stddev in lambda1_e_b_stddev]]
    return np.array(lambda0_stddev), np.array(lambda1_stddev)

def compute_linear_penalty(current_stddev, n_atoms):
    per_atom_thresh = 0.5 * unit.kilojoule_per_mole
    total_thresh = per_atom_thresh * n_atoms
    linear_penalty = np.maximum(0, current_stddev - (total_thresh/kT))
    return linear_penalty

def compute_last_valid_ind(linear_penalty):
    last_valid_ind = np.argmax(np.cumsum(linear_penalty) > 0)
    if last_valid_ind == 0:
        return len(linear_penalty)
    return last_valid_ind

In [4]:
n_atoms = len(atoms)
last_valid_inds = {}
for lam in snapshots:
    lambda0_stddev, lambda1_stddev = calculate_stddev(snapshots[lam])
    current_stddev = (1 - lam) * lambda0_stddev + lam * lambda1_stddev
    print(current_stddev)
    linear_penalty = compute_linear_penalty(current_stddev, n_atoms)
    last_valid_ind = compute_last_valid_ind(linear_penalty)
    print(last_valid_ind)
    last_valid_inds[lam] = last_valid_ind

  0%|          | 0/20 [00:00<?, ?it/s]

[35.54565312 33.71561249 29.49961212 35.12164857 28.42659966 28.84589842
 34.7389601  32.82288587 34.58596583 31.23591895 37.81755263 23.92589236
 30.32409965 34.87454346 31.83022945 29.52869658 32.58677437 27.17266795
 25.70561765 27.1862321 ]
20


  0%|          | 0/20 [00:00<?, ?it/s]

[41.72148378 38.73133952 31.88020362 34.36612729 29.13551649 34.84994173
 36.83820444 33.20671832 32.86367665 24.70991068 36.71820588 30.74188788
 31.87267347 33.8298714  31.75555156 33.35595974 32.21869838 31.34417146
 24.84004115 31.51509683]
20


  0%|          | 0/20 [00:00<?, ?it/s]

...

[47.70792313 32.88582664 33.31627715 29.83241976 29.90668104 32.75142959
 31.6528804  24.8463055  35.83958659 29.91163244 29.59900115 32.31458767
 33.83971691 31.83825474 33.69748973 32.4244972  27.66529539 31.05198847
 32.29890764 31.7087311 ]
20


  0%|          | 0/20 [00:00<?, ?it/s]

[  53.65440113   41.87982487   34.65762627   29.47528006   48.07861583
   55.70499074  151.88327027  192.28808121  211.19113176  355.74530355
  533.62426845  622.72381216  757.59083771  881.66462689  975.34427541
 1132.20267114 1118.37011242 1266.47942677 1293.79382527 1387.36391851]
6


100%|██████████| 20/20 [00:24<00:00,  1.24s/it]

[41.6987579  32.42672286 32.26539073 35.0023438  34.23746625 32.39632299
 31.40239445 33.18019432 35.74752288 34.65752252 26.18046237 33.96806234
 26.49155486 37.52926646 24.57156465 30.76052227 27.32622002 32.80497807
 34.60298191 37.57629227]
20


In [5]:
for lam in sorted(list(last_valid_inds.keys())):
    print('lam={}: usable snapshots: {}'.format(lam, max(0, 10 * last_valid_inds[lam])))

lam=0.0: usable snapshots: 200
lam=0.05: usable snapshots: 200
lam=0.1: usable snapshots: 200
lam=0.15: usable snapshots: 200
lam=0.2: usable snapshots: 200
lam=0.25: usable snapshots: 200
lam=0.3: usable snapshots: 50
lam=0.35: usable snapshots: 200
lam=0.4: usable snapshots: 200
lam=0.45: usable snapshots: 200
lam=0.5: usable snapshots: 200
lam=0.55: usable snapshots: 200
lam=0.6: usable snapshots: 200
lam=0.65: usable snapshots: 200
lam=0.7: usable snapshots: 200
lam=0.75: usable snapshots: 200
lam=0.8: usable snapshots: 200
lam=0.85: usable snapshots: 30
lam=0.9: usable snapshots: 200
lam=0.95: usable snapshots: 60
lam=1.0: usable snapshots: 200


In [ ]:
snapshots = []
N_k = []

max_n_snapshots_per_state = 10

for lam in lambdas_with_usable_samples:
    traj = trajs[lam][5:last_valid_inds[lam]]
    further_thinning = 1
    if len(traj) > max_n_snapshots_per_state:
        further_thinning = int(len(traj) / max_n_snapshots_per_state)
    new_snapshots = list(traj.xyz[::further_thinning] * unit.nanometer)
    snapshots.extend(new_snapshots)
    N_k.append(len(new_snapshots))

N = len(snapshots)
N_k, N

In [ ]:
from collections import namedtuple
RawEnergies = namedtuple('RawEnergies',
                         ['raw_energies_without_dummy_0',
                          'raw_energies_without_dummy_1'
                         ]
                        )
raw_energy_dict = {}
for dcd in dcds:
    print(dcd)
    lam = parse_lambda_from_filename(dcd)
    traj = md.load_dcd(dcd, top=tautomer.ligand_in_water_topology)
    raw_energies_without_dummy_0, raw_energies_without_dummy_1 = compute_endstate_atomic_energy_contributions(traj[::10], model, energy_function)
    raw_energies = RawEnergies(raw_energies_without_dummy_0, raw_energies_without_dummy_1)
    raw_energy_dict[lam] = raw_energies